# Introduction to Julia by Dr. Mohammad Masiur Rahaman (masiurr@iitbbs.ac.in)

## Julia built-in package for symbolic calculation

Symbolics.jl is a fast and modern Computer Algebra System (CAS) for a fast and modern programming 
language (Julia). The goal is to have a high-performance and parallelized symbolic algebra system 
that is directly extendable in the same language as the users.

#### Install the package

In [ ]:
using Pkg; 
Pkg.add("Symbolics")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed ForwardDiff ─────────────── v0.10.30
   Installed Adapt ───────────────────── v3.4.0
   Installed PlotUtils ───────────────── v1.3.0
   Installed ASL_jll ─────────────────── v0.1.3+0
   Installed TimerOutputs ────────────── v0.5.20
   Installed gmsh_jll ────────────────── v4.10.2+0
   Installed ConstructionBase ────────── v1.4.0
   Installed DynamicPolynomials ──────── v0.3.21
   Installed Symbolics ───────────────── v3.2.3
   Installed RuntimeGeneratedFunctions ─ v0.5.3
   Installed OffsetArrays ────────────── v1.12.7
   Installed PDMats ──────────────────── v0.11.16
   Installed Metis ───────────────────── v1.2.0
   Installed StatsBase ───────────────── v0.33.20
   Installed DataAPI ─────────────────── v1.10.0
   Installed Preferences ─────────────── v1.3.0
   Installed CEnum ───────────────────── v0.4.2
   Installed LinearElasticity_jll ────── v5.0.0+0
   Installed HDF5_jll ─────────

#### Load the package

In [1]:
using Symbolics

#### Define one or more unknown variables

Define symbolic variables via the **@variables** macro

**(..)** signifies that the value should be left uncalled.

In [2]:
@variables x y

2-element Vector{Num}:
 x
 y

After defining variables as symbolic, symbolic expressions, which we call a **istree** object, 
can be generated by utilizing Julia expressions.
For example: Given below z is an expression **tree** for "square x and add y".

In [3]:
z = x^2 + y

y + x^2

#### To make an array of symbolic expressions

Simply make an array of symbolic expressions:

In [4]:
A = [x^2 + y 0 2x;
     0       0 2y;
     y^2 + x 0 0]

3×3 Matrix{Num}:
 y + x^2  0  2x
       0  0  2y
 x + y^2  0   0

#### Use normal Julia functions as generators for expressions

In [5]:
function f(u)
  [u[1] - u[3], u[1]^2 - u[2], u[3] + u[2]]
end

f (generic function with 1 method)

In [6]:
f([x, y, z]) # Recall that z = x^2 + y

3-element Vector{Num}:
 x - y - (x^2)
       x^2 - y
      x^2 + 2y

#### Build an array variable and use it to trace the function

In [7]:
@variables u[1:3]
f(u)

3-element Vector{Num}:
   u[1] - u[3]
 u[1]^2 - u[2]
   u[2] + u[3]

## Derivatives

#### To build a differential operator, use *Differential*

For example, the differential operator $D = \frac{\partial}{\partial t}$ is defined as below.

In [8]:
@variables t
D = Differential(t)

(::Differential) (generic function with 2 methods)

In [9]:
z = t + t^2
D(z)         

Differential(t)(t + t^2)

#### Use of *expand_derivatives*

Notice that this hasn't computed anything yet: D is a lazy operator because it lets us symbolically 
represent "The derivative of z with respect to t", which is useful for example when representing 
our favorite thing in the world, differential equations. However, if we want to expand the derivative 
operators, we'd use **expand_derivatives**

In [10]:
expand_derivatives(D(z)) 

1 + 2t

In [12]:
expand_derivatives(D(exp(t^2)-t^2))

2t*exp(t^2) - 2t

#### To get the variable that you are taking the derivative with respect to is accessed with

In [13]:
D.x

t

#### Simplified functions for multivariable calculus. 

 For example, we can compute the Jacobian of an array of expressions like:

In [14]:
Symbolics.jacobian([x + x*y, x^2 + y], [x, y])

2×2 Matrix{Num}:
 1 + y  x
    2x  1

## Simplification and Substitution

- To simplify symbolic expressions, use **simplify** command
- To change values of an expression around, use **substitute** command

In [15]:
B = simplify.([t + t^2 + t + t^2  2t + 4t
               x + y + y + 2t     x^2 - x^2 + y^2])

2×2 Matrix{Num}:
 2t + 2(t^2)   6t
 x + 2t + 2y  y^2

In [16]:
simplify.(substitute.(B, x => y^2))

2×2 Matrix{Num}:
   2t + 2(t^2)   6t
 2t + y^2 + 2y  y^2

In [17]:
simplify.(substitute.(B, (Dict(x => y^2),)))

2×2 Matrix{Num}:
   2t + 2(t^2)   6t
 2t + y^2 + 2y  y^2

#### Interactively evaluate expressions without generating and compiling Julia functions

In [18]:
V = substitute.(B, (Dict(x => 2.0, y => 3.0, t => 4.0),))

2×2 Matrix{Num}:
 40.0  24.0
 16.0   9.0

In [19]:
Symbolics.value.(V)

2×2 Matrix{Float64}:
 40.0  24.0
 16.0   9.0

### Independent and Dependent variables

Define t as a independent variable while x(t) and y(t) as dependent variables.

In [20]:
@variables t x(t) y(t)

3-element Vector{Num}:
    t
 x(t)
 y(t)

In [21]:
z = x + y*t

t*y(t) + x(t)

In [22]:
expand_derivatives(D(z))

t*Differential(t)(y(t)) + y(t) + Differential(t)(x(t))

### Define unrestricted functions

In [23]:
@variables g(..)

1-element Vector{Symbolics.CallWithMetadata{SymbolicUtils.FnType{Tuple, Real}, Base.ImmutableDict{DataType, Any}}}:
 g⋆

Here g is a variable that is a function of other variables. 
Any time that we reference g we have to utilize it as a function:

In [24]:
z = g(x) + g(y)

g(x(t)) + g(y(t))

## Registering Functions

One of the benefits of a one-language Julia symbolic stack is that the primitives are all written in Julia,
and therefore it's trivially extendible from Julia itself. By default, new functions are traced to the 
primitives and the symbolic expressions are written on the primitives. However, we can expand the allowed 
primitives by registering new functions. For example, let's register a new function h:

In [25]:
h(x, y) = x^2 + y
@register h(x, y)

Now when we use h(x, y), it is a symbolic expression and doesn't expand:

In [26]:
h(x, y) + y^2

y(t)^2 + h(x(t), y(t))

In order to use it with the differentiation system, 
we need to register its derivatives. We would do it like this:

In [ ]:
# Derivative w.r.t. the first argument
Symbolics.derivative(::typeof(h), args::NTuple{2,Any}, ::Val{1}) = 2args[1]
# Derivative w.r.t. the second argument
Symbolics.derivative(::typeof(h), args::NTuple{2,Any}, ::Val{2}) = 1

and now it works with the rest of the system:

In [ ]:
Symbolics.derivative(h(x, y) + y^2, x) 

In [ ]:
Symbolics.derivative(h(x, y) + y^2, y) 

## Solving linear system of equations

In [27]:
@variables a b y
eq = a + y ~ b

a + y ~ b

In [28]:
Symbolics.solve_for(eq,y)

b - a

In [29]:
@variables x y
eq1 = 2x + 3y ~ 5
eq2 = 3x - 2y ~ 7
Symbolics.solve_for([eq1,eq2],[x,y])

2-element Vector{Float64}:
 2.3846153846153846
 0.07692307692307693

In [30]:
@variables x y z
eq1 = 2x + 3y + z ~ 5
eq2 = 3x - 2y + 2z ~ 7
eq3 = 4x + 5y + 7z ~ 11
Symbolics.solve_for([eq1,eq2,eq3],[x,y,z])

3-element Vector{Float64}:
 2.25
 0.09375
 0.21874999999999997

## Solving non-linear system of equations

$x^2 + xy +sin(y) =3 $

$x^3 - 4x +sin(2y) =0$

In [31]:
using NLsolve

f(x) = [x[1]^2 + x[1]*x[2] + sin(x[2]) - 3;
    x[1]^3 - 4x[1] + sin(2x[2])]

results = nlsolve(f, [2.0; 0.0], autodiff=:forward)

LoadError: ArgumentError: Package NLsolve not found in current path:
- Run `import Pkg; Pkg.add("NLsolve")` to install the NLsolve package.


In [ ]:
results.zero

## Linear Algebra

In [ ]:
using LinearAlgebra

### Standard Functions

Linear algebra functions in Julia are largely implemented by calling functions from **LAPACK**. 
Sparse factorizations call functions from **SuiteSparse**.

#### Base.:* — Method.

Multiplication operator: x*y*z*... calls this function with all arguments, i.e. *(x, y, z, ...).

In [ ]:
*(2,3,5)

In [ ]:
2*3*5

#### Base.:\ — Method.

Left division operator:  $\(x, y)$ denotes multiplication of y by the inverse of x on the left. 
Gives floating-point results for integer arguments.

In [ ]:
\(2,3)

In [ ]:
2\3

In [ ]:
3 \ 6

In [ ]:
inv(3)*6

In [ ]:
A = [1 2; 3 4]; x = [5, 6];
A\x

In [ ]:
\(A,x)

In [ ]:
inv(A)*x

#### Base.:dot — Function.

**dot()** : Dot product of two vectors

In [ ]:
x = fill(2., (5,5))
y = fill(3., (5,5))
dot(x,y)

Synonym for dot(x, y) is x ⋅ y (where ⋅ can be typed by tab-completing \cdot in the REPL) 

In [ ]:
x⋅y

#### Base.:cross — Function.

**cross()** : cross product of two vectors

In [ ]:
a = [0;1;0]

In [ ]:
b = [0;0;1]

In [ ]:
cross(a,b)

Synonym for cross(a,b) is ×(a,b) (where × can be typed by tab-completing \times in the REPL)

In [ ]:
×(a,b)

#### Base.:Bidiagonal — Function

**Bidiagonal(dv, ev, isupper::Bool)** constructs an upper (isupper=true) or lower (isupper=false) bidiagonal 
matrix using the given diagonal (dv) and off-diagonal (ev) vectors. The result is of type Bidiagonal and 
provides efficient specialized linear solvers, but may be converted into a regular matrix with
convert(Array, _) (or Array(_) for short). ev's length must be one less than the length of dv.

In [ ]:
dv = [1; 2; 3; 4]

In [ ]:
ev = [7; 8; 9]

In [ ]:
Bu = Bidiagonal(dv, ev, 'U')

In [ ]:
Bl = Bidiagonal(dv, ev, 'L')

Construct a Bidiagonal matrix using **Bidiagonal(A,isupper::Bool)** from the main diagonal of A and 
its first super- (if isupper::U) or sub-diagonal (if isupper::L).

In [ ]:
A = [1 1 1 1; 2 2 2 2; 3 3 3 3; 4 4 4 4]

In [ ]:
Bidiagonal(A, :U)

In [ ]:
Bidiagonal(A, :L)